In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/autompg-dataset/auto-mpg.csv


**IMPORTING LIBRARIES**


In [2]:
!pip install tensorflow 
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score


     |████████████████████████████████| 6.8 MB 2.8 MB/s 
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.2.0
    Uninstalling tensorboard-2.2.0:
      Successfully uninstalled tensorboard-2.2.0
ERROR: After October 2020 you may experience errors when installing or updating packages. This is because pip will change the way that it resolves dependency conflicts.

We recommend you use --use-feature=2020-resolver to test your packages with the new resolver before it becomes the default.

pytorch-lightning 0.9.0 requires tensorboard==2.2.0, but you'll have tensorboard 2.3.0 which is incompatible.
You should consider upgrading via the '/opt/conda/bin/python3.7 -m pip install --upgrade pip' command.


**LOAD DATASET**

In [3]:
data=pd.read_csv('../input/autompg-dataset/auto-mpg.csv')
data

,mpg,cylinders,displacement,horsepower,weight,acceleration,model year,origin,car name
0,18.0,8,307.0,130,3504,12.0,70,1,chevrolet chevelle malibu
1,15.0,8,350.0,165,3693,11.5,70,1,buick skylark 320
2,18.0,8,318.0,150,3436,11.0,70,1,plymouth satellite
3,16.0,8,304.0,150,3433,12.0,70,1,amc rebel sst
4,17.0,8,302.0,140,3449,10.5,70,1,ford torino
...,...,...,...,...,...,...,...,...,...
393,27.0,4,140.0,86,2790,15.6,82,1,ford mustang gl
394,44.0,4,97.0,52,2130,24.6,82,2,vw pickup
395,32.0,4,135.0,84,2295,11.6,82,1,dodge rampage
396,28.0,4,120.0,79,2625,18.6,82,1,ford ranger


In [4]:
data=data.drop(columns=['car name'])
data

,mpg,cylinders,displacement,horsepower,weight,acceleration,model year,origin
0,18.0,8,307.0,130,3504,12.0,70,1
1,15.0,8,350.0,165,3693,11.5,70,1
2,18.0,8,318.0,150,3436,11.0,70,1
3,16.0,8,304.0,150,3433,12.0,70,1
4,17.0,8,302.0,140,3449,10.5,70,1
...,...,...,...,...,...,...,...,...
393,27.0,4,140.0,86,2790,15.6,82,1
394,44.0,4,97.0,52,2130,24.6,82,2
395,32.0,4,135.0,84,2295,11.6,82,1
396,28.0,4,120.0,79,2625,18.6,82,1


Here we convert the origin country which is a categorical column to numerical taking UK=1, EUROPE=2 and USA=3. 

In [5]:
origin=data.pop('origin')
data['uk']=(origin==1)*1.0
data['europe']=(origin==2)*2.0
data['usa']=(origin==3)*3.0
data

,mpg,cylinders,displacement,horsepower,weight,acceleration,model year,uk,europe,usa
0,18.0,8,307.0,130,3504,12.0,70,1.0,0.0,0.0
1,15.0,8,350.0,165,3693,11.5,70,1.0,0.0,0.0
2,18.0,8,318.0,150,3436,11.0,70,1.0,0.0,0.0
3,16.0,8,304.0,150,3433,12.0,70,1.0,0.0,0.0
4,17.0,8,302.0,140,3449,10.5,70,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...
393,27.0,4,140.0,86,2790,15.6,82,1.0,0.0,0.0
394,44.0,4,97.0,52,2130,24.6,82,0.0,2.0,0.0
395,32.0,4,135.0,84,2295,11.6,82,1.0,0.0,0.0
396,28.0,4,120.0,79,2625,18.6,82,1.0,0.0,0.0


Splitted the complete dataset into train and test data in the fraction of 80:20

In [6]:
train_data=data.sample(frac=0.8, random_state=0)
test_data=data.drop(train_data.index)
print(train_data)
print(test_data)

      mpg  cylinders  displacement horsepower  weight  acceleration  \
65   14.0          8         351.0        153    4129          13.0   
132  25.0          4         140.0         75    2542          17.0   
74   13.0          8         302.0        140    4294          16.0   
78   21.0          4         120.0         87    2979          19.5   
37   18.0          6         232.0        100    3288          15.5   
..    ...        ...           ...        ...     ...           ...   
207  20.0          4         130.0        102    3150          15.7   
279  29.5          4          98.0         68    2135          16.6   
227  19.0          6         225.0        100    3630          17.7   
148  26.0          4         116.0         75    2246          14.0   
143  26.0          4          97.0         78    2300          14.5   

     model year   uk  europe  usa  
65           72  1.0     0.0  0.0  
132          74  1.0     0.0  0.0  
74           72  1.0     0.0  0.0  
78 

Basic Statistics of the train dataset

In [7]:
train_data_state=train_data.describe()
train_data_state=train_data_state.transpose()
train_data_state
#train_data['horsepower'].describe()


,count,mean,std,min,25%,50%,75%,max
mpg,318.0,23.590566,7.913617,10.0,17.125,22.75,29.00,46.6
cylinders,318.0,5.427673,1.682941,3.0,4.000,4.00,6.00,8.0
displacement,318.0,193.061321,103.812742,70.0,100.250,151.00,259.50,455.0
weight,318.0,2963.823899,844.749805,1613.0,2219.250,2792.50,3571.25,5140.0
acceleration,318.0,15.595912,2.796282,8.0,13.900,15.50,17.30,24.8
model year,318.0,75.946541,3.705266,70.0,73.000,76.00,79.00,82.0
uk,318.0,0.641509,0.480313,0.0,0.000,1.00,1.00,1.0
europe,318.0,0.327044,0.740848,0.0,0.000,0.00,0.00,2.0
usa,318.0,0.584906,1.190403,0.0,0.000,0.00,0.00,3.0


In [8]:
a=train_data['horsepower']
a

65     153
132     75
74     140
78      87
37     100
      ... 
207    102
279     68
227    100
148     75
143     78
Name: horsepower, Length: 318, dtype: object

In [9]:
a.unique()

array(['153', '75', '140', '87', '100', '69', '65', '90', '150', '110',
       '?', '88', '54', '145', '80', '82', '105', '190', '78', '97',
       '112', '70', '116', '66', '84', '92', '72', '96', '155', '71',
       '198', '220', '165', '180', '107', '67', '76', '83', '130', '170',
       '200', '95', '148', '215', '61', '60', '46', '74', '120', '63',
       '225', '52', '129', '85', '175', '98', '86', '89', '158', '68',
       '93', '62', '230', '103', '125', '115', '53', '149', '58', '152',
       '138', '48', '210', '108', '139', '208', '167', '94', '64', '160',
       '77', '122', '113', '102'], dtype=object)

Replacing '?' in horsepower with 0

In [10]:
a=a.replace('?',0)
a.unique()

array(['153', '75', '140', '87', '100', '69', '65', '90', '150', '110', 0,
       '88', '54', '145', '80', '82', '105', '190', '78', '97', '112',
       '70', '116', '66', '84', '92', '72', '96', '155', '71', '198',
       '220', '165', '180', '107', '67', '76', '83', '130', '170', '200',
       '95', '148', '215', '61', '60', '46', '74', '120', '63', '225',
       '52', '129', '85', '175', '98', '86', '89', '158', '68', '93',
       '62', '230', '103', '125', '115', '53', '149', '58', '152', '138',
       '48', '210', '108', '139', '208', '167', '94', '64', '160', '77',
       '122', '113', '102'], dtype=object)

As we see in the basic statistics of dataset, all the columns in different scales, here we normalize data using z-score normalization.

In [11]:
def norm(x):
    return(x-train_data_state['mean'])/train_data_state['std']
normed_train_data=norm(train_data)
normed_train_data

,acceleration,cylinders,displacement,europe,horsepower,model year,mpg,uk,usa,weight
65,-0.928344,1.528471,1.521380,-0.441445,NaN,-1.065117,-1.211907,0.746369,-0.491351,1.379315
132,0.502127,-0.848320,-0.511125,-0.441445,NaN,-0.525344,0.178102,0.746369,-0.491351,-0.499348
74,0.144509,1.528471,1.049377,-0.441445,NaN,-1.065117,-1.338271,0.746369,-0.491351,1.574639
78,1.396171,-0.848320,-0.703780,2.258163,NaN,-1.065117,-0.327355,-1.335607,-0.491351,0.017965
37,-0.034300,0.340075,0.375086,-0.441445,NaN,-1.335003,-0.706449,0.746369,-0.491351,0.383754
...,...,...,...,...,...,...,...,...,...,...
207,0.037224,-0.848320,-0.607453,2.258163,NaN,0.014428,-0.453720,-1.335607,-0.491351,0.220392
279,0.359080,-0.848320,-0.915700,-0.441445,NaN,0.554200,0.746742,-1.335607,2.028805,-0.981147
227,0.752459,0.340075,0.307657,-0.441445,NaN,0.284314,-0.580084,0.746369,-0.491351,0.788608
148,-0.570726,-0.848320,-0.742311,2.258163,NaN,-0.525344,0.304467,-1.335607,-0.491351,-0.849747


In [12]:
normed_train_data=normed_train_data.drop(columns=['horsepower'])
normed_train_data

,acceleration,cylinders,displacement,europe,model year,mpg,uk,usa,weight
65,-0.928344,1.528471,1.521380,-0.441445,-1.065117,-1.211907,0.746369,-0.491351,1.379315
132,0.502127,-0.848320,-0.511125,-0.441445,-0.525344,0.178102,0.746369,-0.491351,-0.499348
74,0.144509,1.528471,1.049377,-0.441445,-1.065117,-1.338271,0.746369,-0.491351,1.574639
78,1.396171,-0.848320,-0.703780,2.258163,-1.065117,-0.327355,-1.335607,-0.491351,0.017965
37,-0.034300,0.340075,0.375086,-0.441445,-1.335003,-0.706449,0.746369,-0.491351,0.383754
...,...,...,...,...,...,...,...,...,...
207,0.037224,-0.848320,-0.607453,2.258163,0.014428,-0.453720,-1.335607,-0.491351,0.220392
279,0.359080,-0.848320,-0.915700,-0.441445,0.554200,0.746742,-1.335607,2.028805,-0.981147
227,0.752459,0.340075,0.307657,-0.441445,0.284314,-0.580084,0.746369,-0.491351,0.788608
148,-0.570726,-0.848320,-0.742311,2.258163,-0.525344,0.304467,-1.335607,-0.491351,-0.849747


In [13]:
normed_train_data['horsepower']=a
normed_train_data

,acceleration,cylinders,displacement,europe,model year,mpg,uk,usa,weight,horsepower
65,-0.928344,1.528471,1.521380,-0.441445,-1.065117,-1.211907,0.746369,-0.491351,1.379315,153
132,0.502127,-0.848320,-0.511125,-0.441445,-0.525344,0.178102,0.746369,-0.491351,-0.499348,75
74,0.144509,1.528471,1.049377,-0.441445,-1.065117,-1.338271,0.746369,-0.491351,1.574639,140
78,1.396171,-0.848320,-0.703780,2.258163,-1.065117,-0.327355,-1.335607,-0.491351,0.017965,87
37,-0.034300,0.340075,0.375086,-0.441445,-1.335003,-0.706449,0.746369,-0.491351,0.383754,100
...,...,...,...,...,...,...,...,...,...,...
207,0.037224,-0.848320,-0.607453,2.258163,0.014428,-0.453720,-1.335607,-0.491351,0.220392,102
279,0.359080,-0.848320,-0.915700,-0.441445,0.554200,0.746742,-1.335607,2.028805,-0.981147,68
227,0.752459,0.340075,0.307657,-0.441445,0.284314,-0.580084,0.746369,-0.491351,0.788608,100
148,-0.570726,-0.848320,-0.742311,2.258163,-0.525344,0.304467,-1.335607,-0.491351,-0.849747,75


In [14]:
labels_of_train_data=normed_train_data['mpg']
features_of_train_data=normed_train_data.drop(columns=['mpg'])
print(labels_of_train_data)
print(features_of_train_data)

65    -1.211907
132    0.178102
74    -1.338271
78    -0.327355
37    -0.706449
         ...   
207   -0.453720
279    0.746742
227   -0.580084
148    0.304467
143    0.304467
Name: mpg, Length: 318, dtype: float64
     acceleration  cylinders  displacement    europe  model year        uk  \
65      -0.928344   1.528471      1.521380 -0.441445   -1.065117  0.746369   
132      0.502127  -0.848320     -0.511125 -0.441445   -0.525344  0.746369   
74       0.144509   1.528471      1.049377 -0.441445   -1.065117  0.746369   
78       1.396171  -0.848320     -0.703780  2.258163   -1.065117 -1.335607   
37      -0.034300   0.340075      0.375086 -0.441445   -1.335003  0.746369   
..            ...        ...           ...       ...         ...       ...   
207      0.037224  -0.848320     -0.607453  2.258163    0.014428 -1.335607   
279      0.359080  -0.848320     -0.915700 -0.441445    0.554200 -1.335607   
227      0.752459   0.340075      0.307657 -0.441445    0.284314  0.746369   
148  

Train test split

In [15]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(features_of_train_data,labels_of_train_data, test_size=0.3)

BUILD THE MODEL-
Building of a feed forward neural network with Two hidden layers and one output layer with 16 units in the hidden layers and using relu activation function as the non-linear activation function. Optimizer used in this model is RMSprop which is used in general for the model of regression.

In [16]:
def build_model():
    model=keras.Sequential([layers.Dense(16, activation='relu'),layers.Dense(16, activation='relu'),
                                                 layers.Dense(1)])
    optimizer=tf.keras.optimizers.RMSprop(0.001)
    model.compile(loss='mse', optimizer=optimizer, metrics=['mae','mse'])
    return model

In [17]:
model=build_model()

In [18]:
#def baseline_model():
	# create model
#	model = Sequential()
#	model.add(Dense(13, input_dim=13, kernel_initializer='normal', activation='relu'))
#	model.add(Dense(1, kernel_initializer='normal'))
	# Compile model
#	model.compile(loss='mean_squared_error', optimizer='adam')
#	return model

In [19]:
 #evaluate model
#estimator = KerasRegressor(build_fn=baseline_model, epochs=100, batch_size=5, verbose=0)

In [20]:
#kfold = KFold(n_splits=10)
#results = cross_val_score(estimator, x_train, y_train, cv=kfold)
#print("Baseline: %.2f (%.2f) MSE" % (results.mean(), results.std()))

Converting the Features and labels of the dataset to numpy array to feed into the network.

In [21]:
X=x_train.to_numpy(dtype=np.float)
Y=y_train.to_numpy(dtype=np.float)

Train the model

In [22]:
model.fit(X,Y, epochs=1000)

Epoch 1/1000
7/7 [==============================] - 0s 1ms/step - loss: 1.0244 - mae: 0.7683 - mse: 1.0244
Epoch 2/1000
7/7 [==============================] - 0s 1ms/step - loss: 0.5635 - mae: 0.5525 - mse: 0.5635
Epoch 3/1000
7/7 [==============================] - 0s 1ms/step - loss: 0.5971 - mae: 0.5865 - mse: 0.5971
Epoch 4/1000
7/7 [==============================] - 0s 1ms/step - loss: 0.5872 - mae: 0.5639 - mse: 0.5872
Epoch 5/1000
7/7 [==============================] - 0s 1ms/step - loss: 0.5620 - mae: 0.5684 - mse: 0.5620
Epoch 6/1000
7/7 [==============================] - 0s 1ms/step - loss: 0.4931 - mae: 0.5026 - mse: 0.4931
Epoch 7/1000
7/7 [==============================] - 0s 1ms/step - loss: 0.4499 - mae: 0.4753 - mse: 0.4499
Epoch 8/1000
7/7 [==============================] - 0s 1ms/step - loss: 0.4340 - mae: 0.4777 - mse: 0.4340
Epoch 9/1000
7/7 [==============================] - 0s 1ms/step - loss: 0.5389 - mae: 0.5517 - mse: 0.5389
Epoch 10/1000
7/7 [==================

In [23]:
X1=x_test.to_numpy(dtype=np.float)
Y1=y_test.to_numpy(dtype=np.float)

MAKE PREDICTIONS

In [24]:
test_predictions=model.predict(X1)
test_predictions

array([[ 4.4103855e-01],
       [ 2.0161879e+00],
       [-7.8743142e-01],
       [-5.9306711e-01],
       [ 6.4397436e-01],
       [ 9.5832068e-01],
       [-8.3321020e-02],
       [ 1.2826593e-01],
       [-1.0261313e+00],
       [-1.2900134e+00],
       [ 7.0642859e-01],
       [ 7.4321991e-01],
       [ 4.6731979e-01],
       [-3.2051939e-01],
       [ 1.5313551e-04],
       [ 1.2638137e+00],
       [-8.6045712e-01],
       [ 4.9860793e-01],
       [ 7.7067536e-01],
       [-3.7284487e-01],
       [ 3.5856079e-02],
       [ 1.5395552e+00],
       [ 6.6316706e-01],
       [-2.7290910e-01],
       [-9.3737763e-01],
       [-5.1387161e-01],
       [ 4.8839401e-02],
       [-1.0234944e+00],
       [-1.7821114e-01],
       [-1.2602197e+00],
       [-6.3589782e-01],
       [ 1.9969350e+00],
       [ 1.8305284e+00],
       [ 1.0797515e+00],
       [ 1.0152109e-01],
       [ 5.7549840e-01],
       [-5.1396388e-01],
       [-4.1891092e-01],
       [-9.5155662e-01],
       [-3.5094017e-01],


EVALUATE THE MODEL USING MEAN SQUARED ERROR

In [25]:
loss,mae,mse=model.evaluate(X1,Y1,verbose=0)
print('Testing mean squared error:MPG', format(mse))

Testing mean squared error:MPG 0.14804008603096008


**As we can see the mean squared error of train set and test set is 0.10 and 0.16, this shows that the model is perfroming well. However it can do better with further optimizations.**

**IF YOU FOUND THIS NOTEBOOK USEFULL THEN PLEASE UPVOTE✌**